# Import
Add "from gensim import models"

In [1]:
#required imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import string
import collections
import re
import pycountry
from gensim import models, corpora
from os import path
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import bigrams
from nltk import ngrams
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.sentiment import SentimentAnalyzer
from nltk.corpus import opinion_lexicon
from nltk.tokenize import treebank
import sentiment as senth


#local helper file import
import nlp_helper as nlph

C:\Users\Alain\Anaconda3\lib\site-packages\gensim\utils.py:840: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\Alain\Anaconda3\lib\site-packages\gensim\utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")
C:\Users\Alain\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
raws = pd.read_csv('hillary-clinton-emails/Emails.csv',usecols=['ExtractedSubject','ExtractedBodyText','RawText'])
raws.head()

,ExtractedSubject,ExtractedBodyText,RawText
0,FW: Wow,NaN,UNCLASSIFIED\r\nU.S. Department of State\r\nCa...
1,NaN,"B6\r\nThursday, March 3, 2011 9:45 PM\r\nH: La...",UNCLASSIFIED\r\nU.S. Department of State\r\nCa...
2,Re: Chris Stevens,Thx,UNCLASSIFIED\r\nU.S. Department of State\r\nCa...
3,FVV: Cairo Condemnation - Final,NaN,UNCLASSIFIED\r\nU.S. Department of State\r\nCa...
4,NaN,"H <hrod17@clintonemail.com>\r\nFriday, March 1...",B6\r\nUNCLASSIFIED\r\nU.S. Department of State...


In [3]:
raws.drop(['RawText'], axis= 1,inplace=True)

In [4]:
raws.head()

,ExtractedSubject,ExtractedBodyText
0,FW: Wow,NaN
1,NaN,"B6\r\nThursday, March 3, 2011 9:45 PM\r\nH: La..."
2,Re: Chris Stevens,Thx
3,FVV: Cairo Condemnation - Final,NaN
4,NaN,"H <hrod17@clintonemail.com>\r\nFriday, March 1..."


In [5]:
body = pd.DataFrame()
body['text'] = raws.apply(nlph.concat_subj_txt, axis=1)

In [6]:
stop = set(stopwords.words('english'))
stop.update(string.punctuation) #Remove ponctuation
stop.update(['nan', '\'s', '--', '``', 'w', 'fw', 'n\'t', '\'m', 'also', 'thx', 'fyi', 'pls', '\'\'', '-', '—', 'pm', '•'])

In [11]:
body_tokenized = [[i for i in word_tokenize(text.lower())if i not in stop] for text in body['text']]

## Part 3 Topic Model

We create a dictionary and the corpus using the tokenized body of the emails.

In [8]:
dictionary = corpora.Dictionary(body_tokenized)
corpus = [dictionary.doc2bow(text) for text in body_tokenized]

We then build the LDA Model with number of topics between 5 to 50 and print each topic with 7 words to compare what number of topic returns meaningful topics at first sight  :

In [9]:
for i in range(5,51,5):
    lda = models.LdaModel(corpus, id2word=dictionary, num_topics=i)
    print("Number of topics : ", i , " / Number of words per topic : 7 \n")
    model = lda.show_topics(num_topics = i, num_words = 7, formatted=False)
    i = 0
    for topic in model:
        wordlist = topic[1]
        i+=1
        print("Topic " + str(i) + " has words :", end=" ")
        for word, prob in wordlist:
            print(word, end=", ")
        print("")
    print("\n-------------------------------------------------------------\n")

Number of topics :  5  / Number of words per topic : 7 

Topic 1 has words : office, state, call, secretary, department, time, room, 
Topic 2 has words : obama, one, president, new, would, party, said, 
Topic 3 has words : obama, said, would, state, house, senate, new, 
Topic 4 has words : 2010, h, secretary, office, 2, state.gov, 1, 
Topic 5 has words : would, call, one, get, new, women, us, 

-------------------------------------------------------------

Number of topics :  10  / Number of words per topic : 7 

Topic 1 has words : call, 2010, h, get, today, huma, tomorrow, 
Topic 2 has words : new, state, time, american, president, said, would, 
Topic 3 has words : would, women, one, said, president, see, time, 
Topic 4 has words : 2010, h, state.gov, cheryl, mills, b6, j, 
Topic 5 has words : state, right-wing, house, book, branch, lona, office, 
Topic 6 has words : 2, 1, 3, call, 4, 2010, would, 
Topic 7 has words : party, one, new, 1.4, 2010, h, us, 
Topic 8 has words : obama, sai